# TP1 : Algorithme EM et modèle de mélange gaussien

## Introduction à l'estimation statistique - G3 SDIA

L'objectif de ce TP est d'implémenter l'algorithme EM pour estimer par maximum de vraisemblance les paramètres d'un modèle de mélange gaussien.

On utilisera le dataset *Old Faithful*, qui décrit 272 éruptions du geyser appelé Old Faithful du parc national de Yellowstone aux États-Unis. Chaque observation est constituée de 2 variables : le temps d'attente avant l'éruption (en minutes) et la durée de l'éruption (en minutes).

### Instructions

1. Renommer votre notebook sous la forme `tp1_Nom1_Nom2.ipynb`, et inclure le nom du binôme dans le notebook. 

2. Votre code, ainsi que toute sortie du code, doivent être commentés !

3. Déposer votre notebook sur Moodle dans la section prévue à cet effet avant la date limite : 15 Octobre 2023, 23h59.

In [12]:
# Import necessary libraries
import numpy as np
import scipy.stats as ss
import seaborn
from matplotlib import pyplot as plt
import plotly.graph_objects as go


**Q1**. Charger le dataset, normaliser puis visualiser les données. Commenter.

In [13]:
# Load the "Old Faithful" dataset from the Seaborn library
df = seaborn.load_dataset('geyser')
display(df)
X = df[['duration', 'waiting']].values
N, D = X.shape
print(df.describe())

# Standardize data to avoid numerical instabilities

# Standardiser X
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X_standardized = (X - mean) / std

print(X_standardized[:5])

# Show data : nuage de points

fig = go.Figure(data=go.Scatter(x=X_standardized[:, 0], y=X_standardized[:, 1], mode='markers'))

# Mise en forme du graphique
fig.update_layout(title="Standardized Geyser Data",
                  xaxis_title="Duration",
                  yaxis_title="Waiting",
                  showlegend=False)

fig.show()


,duration,waiting,kind
0,3.600,79,long
1,1.800,54,short
2,3.333,74,long
3,2.283,62,short
4,4.533,85,long
...,...,...,...
267,4.117,81,long
268,2.150,46,short
269,4.417,90,long
270,1.817,46,short


         duration     waiting
count  272.000000  272.000000
mean     3.487783   70.897059
std      1.141371   13.594974
min      1.600000   43.000000
25%      2.162750   58.000000
50%      4.000000   76.000000
75%      4.454250   82.000000
max      5.100000   96.000000
[[ 0.09849886  0.59712344]
 [-1.48145856 -1.24518118]
 [-0.13586149  0.22866251]
 [-1.05750332 -0.6556437 ]
 [ 0.91744345  1.03927655]]



----- Votre réponse ici -----








**Q2**. On note $\mathbf{x}_1,...,\mathbf{x}_n$ les données. On souhaite les modéliser par un modèle de mélange gaussien à $K$ composantes.

Écrire une fonction permettant de calculer la log-vraisemblance :
$$ \log \mathcal{L}(\theta;\mathbf{x}_1,...,\mathbf{x}_n) = \sum_{i=1}^n \log \left( \sum_{k=1}^K \pi_k \frac{1}{2 \pi \text{det}(\boldsymbol{\Sigma}_k)^{1/2}} \exp \left( \frac{1}{2} (\mathbf{x}_i - \boldsymbol{\mu}_k)^{\top} \boldsymbol{\Sigma}_k^{-1} (\mathbf{x}_i - \boldsymbol{\mu}_k) \right) \right), $$
avec $\theta = \{ \boldsymbol{\mu_1}, ..., \boldsymbol{\mu_k}, \boldsymbol{\Sigma}_1, ..., \boldsymbol{\Sigma}_k, \pi_1, ..., \pi_k \}$.

On pourra utiliser la fonction $\texttt{multivariate} \_ \texttt{normal.pdf}$ de la librairie scipy.stats.

In [15]:
from scipy.stats import multivariate_normal

def log_likelihood(X, weights, means, covariances):
    """
    Calcule la log-vraisemblance pour un mélange de gaussiennes.

    Paramètres:
    - X: les données (nombre d'échantillons, nombre de caractéristiques)
    - weights: les poids des composants gaussiens
    - means: les moyennes des composants gaussiens (nombre de composants, nombre de caractéristiques)
    - covariances: les covariances des composants gaussiens (nombre de composants, nombre de caractéristiques, nombre de caractéristiques)

    Retourne:
    - La log-vraisemblance des données étant générées par le mélange de gaussiennes.
    """
    n_samples, n_features = X.shape
    n_components = len(weights)
    
    # Calculer la probabilité de chaque point de données pour chaque composant gaussien
    probabilities = np.zeros((n_samples, n_components))
    for k in range(n_components):
        probabilities[:, k] = weights[k] * multivariate_normal.pdf(X, mean=means[k], cov=covariances[k])
    
    # Somme sur les composants pour obtenir la proba totale pour chaque point de données
    total_proba = np.sum(probabilities, axis=1)
    
    return np.sum(np.log(total_proba))

In [16]:
# test 
weights = [0.5, 0.5]
means = [[0, 0], [1, 1]]  # Ces moyennes sont choisies arbitrairement
covariances = [np.eye(2), np.eye(2)]  # Matrices d'identité pour la simplicité

ll = log_likelihood(X_standardized, weights, means, covariances)
print(f"Log-vraisemblance: {ll}")

Log-vraisemblance: -790.8733976257811


**Q3**. Écrire une fonction qui implémente l'algorithme EM dans ce modèle, prenant pour arguments les données, le nombre de composantes $K$, et le nombre d'itérations de l'algorithme $N_{\text{iter}}$. Cette fonction retournera un tableau de taille $N_{\text{iter}} + 1$ contenant l'évolution des valeurs de la log-vraisemblance, ainsi que les valeurs finales des paramètres.

Initialisation des paramètres :
- Pour les moyennes, les $K$ premières observations du dataset ;
- Pour les matrices de covariances, la matrice identité ;
- $\pi_k = 1/K$.

In [ ]:
def EM_algorithm_v1(X, K, Niter):
    N, D = X. shape
    
    # Initialize parameters

    # # # # # # # # #
    # YOUR CODE HERE #
    # # # # # # # # #

    for i in range(0,Niter):
        # E-STEP
        
        # # # # # # # # #
        # YOUR CODE HERE #
        # # # # # # # # #
        
        # M-STEP
        
        # # # # # # # # #
        # YOUR CODE HERE #
        # # # # # # # # #
        
    return # YOUR CODE HERE #

**Q4**. Faire tourner l'algorithme avec $K = 2$ et $N_{\text{iter}} = 50$.

Afficher l'évolution de la log-vraisemblance en fonction des itérations. Commenter.

Sur une même figure, afficher le dataset et représenter les estimations des deux lois normales du mélange à l'aide d'un *contour plot*. On pourra utiliser la fonction $\texttt{plt.contour}$.

In [ ]:
K = 2
Niter = 50

# # # # # # # # #
# YOUR CODE HERE #
# # # # # # # # #

----- Votre réponse ici -----

**Q5**. On souhaite maintenant étudier l'influence de l'initialisation sur les résultats. Modifier la fonction implémentant l'algorithme EM en y rajoutant un argument pour la graine aléatoire. Les paramètres seront maintenant initialisés de la manière suivante :
- $\boldsymbol{\mu}_k \sim \mathcal{N}(\mathbf{0},\mathbf{I}_2)$ ;
- $[\pi_1, ..., \pi_K]^{\top} \sim \text{Dirichlet}([1, ..., 1]^{\top})$ ;
- On gardera l'initialisation des matrices de covariance à la matrice identité.

In [ ]:
def EM_algorithm_v2(X, K, Niter, seed):
    N, D = X. shape
    
    # Initialize parameters
    np.random.seed(seed)

    # # # # # # # # #
    # YOUR CODE HERE #
    # # # # # # # # #

    for i in range(0,Niter):
        # E-STEP
        
        # # # # # # # # #
        # YOUR CODE HERE #
        # # # # # # # # #
        
        # M-STEP
        
        # # # # # # # # #
        # YOUR CODE HERE #
        # # # # # # # # #
        
    return # YOUR CODE HERE #

**Q6**. On choisit maintenant $K=3$. Représenter l'évolution de la log-vraisemblance pour 10 graines aléatoires différentes. Commenter.

Afficher deux cas où la solution retournée par l'algorithme EM est visuellement différente. Commenter.

Quelle estimation de paramètres doit-on choisir ?

In [ ]:
K = 3

# # # # # # # # #
# YOUR CODE HERE #
# # # # # # # # #


----- Votre réponse ici -----

**Q7**. On cherche maintenant à choisir la valeur optimale de $K$. Pour cela, on aimerait pouvoir comparer la vraisemblance des modèles obtenus avec différentes valeurs de $K$.

Cela peut se faire au travers d'un critère de sélection de modèle. Dans ce TP, nous étudierons le critère dit BIC :
$$ \text{BIC}(m) = k(m) \log(n) - 2 \log \mathcal{L}(m),$$
où $m$ est un modèle (ici donné par une valeur de $K$), $k_m$ est le nombre de paramètres libres dans le modèle, $n$ le nombre d'échantillons, et $\mathcal{L}_m$ le maximum de la fonction de vraisemblance de le modèle $m$. On sélectionne le modèle avec le plus faible BIC.

Montrer que $$k(m) = \frac{K}{2} (D+1)(D+2) - 1.$$

Comparer les valeurs de $K$ allant de 1 à 6. Quel est le modèle optimal d'après le critère BIC ?

In [ ]:
# # # # # # # # #
# YOUR CODE HERE #
# # # # # # # # #


----- Votre réponse ici -----

**Question bonus**. Expliquer comment l'algorithme EM peut-être utilisé pour du clustering.

> L'algorithme EM est une méthode itérative utilisée pour trouver des estimations de maximum de vraisemblance dans les modèles statistiques où le modèle dépend de variables cachées non observées. Il est particulièrement utile lorsque les données peuvent être modélisées comme un mélange de plusieurs distributions, comme c'est le cas avec le mélange de gaussiennes.
>
> Comme nous avons pu le voir dans ce TP, une des applications les plus courantes de l'EM est le mélange de gaussiennes, qui est souvent utilisé pour le clustering. Voici une explication détaillée de la manière dont l'algorithme EM peut être utilisé pour le clustering :
>
> Supposons que nos données soient générées à partir de plusieurs distributions gaussiennes (clusters). Chaque point de données appartient à l'un de ces clusters, mais nous ne connaissons pas cette appartenance (c'est notre variable cachée). 
>
> On pourra alors choisir un nombre K de clusters (gaussiennes) et initialiser les paramètres de chaque gaussienne(moyenne, covariance) et poids de chaque cluster.
>
> L'étape d'Expectation permet de calculer la probabilité que chaque observation provienne d'un cluster, en se basant sur les paramètres actuels de chaque gaussienne.
>
> L'étape de Maximization permet de mettre à jour les paramètres de chaque gaussienne en fonction des poids calculés à l'étape d'Expectation.
>
> Enfin, il suffira de répéter ces deux étapes de manière itérative jusqu'à ce que la log-vraisemblance des données sous le modèle actuel converge, ou jusqu'à ce que d'autres critères d'arrêt soient satisfaits. C'est la convergence. En d'autres termes, chaque point de données sera attribué au cluster pour lequel il a le poids le plus élevé. 
> Cependant, il faut faire attention car cet algorithme est sensible aux conditions initiales, peut converger vers des optima locaux, et le choix du nombre de clusters K doit être fait a priori. 

